In [40]:
import numpy as np
import pandas as pd

In [41]:
num_simu = 10 # number of total simulations, smaller number for testing, larger number for data generation.
np.random.seed(69420)

To begin, we set up and make some assumptions of our model. In this particular case:
- We have one singular flock that has both chickens and ducks in it.
- Within this flock, we categorise the chickens into unvaccinated chickens, sentinel chickens (used to detect outbreaks), and vaccinated chickens. We assume that the sentinel chickens are unvaccinated (thus fully susceptible to HPAI), and the vaccinated chickens cannot be infected at all.
- We fix the value of total chicken population to be 3000, with 30 sentinels. We may vary the total duck, vaccinated chicken population, and testing period for our exploratory analysis.

In [42]:
######## Changable Values ########
# set number of species and flocks
num_flocks = 1 # One single flock, need to reflect in beta, sigma, and gamma 
num_species = 4 # chicken, sentinel chicken, vaccinated chicken, duck

tot_chicken_popul = 3000 # total chicken population
tot_duck_popul = 3000 # <---- set total duck population (vary between [300, 1500, 3000, 5000])
vaccinated = 0 # <---- choose how many chickens to be vaccinated (vary between [0, 1500, 2250, 2700])

######## Surveillance Strategy ########
surveillance = 30 # how many chickens are sentinel birds / how many chickens to randomly sample
testing_period = 7 # how many days to test the sentinel birds / do random testing. 

We will consider an SEEIIR model where we will split each infected bird into a symptomatic or asymptomatic case. The ability to infect, the latency period, and infectious period can different. Chickens are more likely to be symptomatic while ducks are more likely to be asymptomatic. Since a lot of parameters are unknown in the literature, we assume that a symptomatic bird (whether chicken or duck) will infect a bird of the same species in the same way, and an equal or lower rate for a different species, because birds of the same species could be more willing to mingle. Similar argument applies to asymptomatic birds.

In [43]:
######## Changable Values ########
# Here we may adjust the key parameters for the simulation
same_species_symptomatic_infection_rate = 1.13
same_species_asymptomatic_infection_rate = 1.07
different_species_symptomatic_infection_rate = 0.3 # vary between [0.3, 0.6, 1.13]
different_species_asymptomatic_infection_rate = 0.25 # vary between [0.25, 0.5, 1.07]

chicken_symptomatic_latency_period = 1
duck_symptomatic_latency_period = 1
chicken_asymptomatic_latency_period = 1
duck_asymptomatic_latency_period = 1

chicken_symptomatic_infectious_period = 3.2
duck_symptomatic_infectious_period = 3.2
chicken_asymptomatic_infectious_period = 4.9
duck_asymptomatic_infectious_period = 4.9

chicken_symptomatic_prob = 0.95
duck_symptomatic_prob = 0.05

In [44]:
# initialise the infection rate tensor
beta_S = np.zeros((num_flocks, num_species, num_flocks, num_species))

beta_S[:, :2, :, :2] = same_species_symptomatic_infection_rate # within-chicken infection
beta_S[:, :2, :, 3] = different_species_symptomatic_infection_rate # chicken-to-duck infection
beta_S[:, 3, :, :2] = different_species_asymptomatic_infection_rate # duck-to-chicken infection
beta_S[:, 3, :, 3] = same_species_symptomatic_infection_rate # within-duck infection

beta_A = np.zeros((num_flocks, num_species, num_flocks, num_species))

beta_A[:, :2, :, :2] = same_species_asymptomatic_infection_rate # within-chicken infection
beta_A[:, :2, :, 3] = different_species_asymptomatic_infection_rate # chicken-to-duck infection
beta_A[:, 3, :, :2] = different_species_asymptomatic_infection_rate # duck-to-chicken infection
beta_A[:, 3, :, 3] = same_species_asymptomatic_infection_rate # within-duck infection

# latency and infectious period
latency_period_S = np.array([chicken_symptomatic_latency_period, chicken_symptomatic_latency_period, 1, duck_symptomatic_latency_period])
sigma_S = 1 / latency_period_S
latency_period_A = np.array([chicken_asymptomatic_latency_period, chicken_asymptomatic_latency_period, 1, duck_asymptomatic_latency_period])
sigma_A = 1 / latency_period_A

infectious_period_S = np.array([chicken_symptomatic_infectious_period, chicken_symptomatic_infectious_period, 1, duck_symptomatic_infectious_period])
gamma_S = 1 / infectious_period_S
infectious_period_A = np.array([chicken_asymptomatic_infectious_period, chicken_asymptomatic_infectious_period, 1, duck_asymptomatic_infectious_period])
gamma_A = 1 / infectious_period_A

# probability of displaying symptoms
p_S = np.array([chicken_symptomatic_prob, chicken_symptomatic_prob, 0, duck_symptomatic_prob])
p_A = np.ones(num_species) - p_S



We set up the initial population accordingly. We assume that one symptomatic chicken is in the flock at the beginning.

In [45]:
######## set initial conditions ######## 

# set initial conditions 

# the following convention will be used: first dimension will represent which flock, second is which species, third is the compartment.
init_val = np.zeros((num_flocks, num_species, 6)) # Six possible compartment: S, E_S, E_A, I_S, I_A, R

# first let all birds to start susceptible, also choose population size here.
init_val[0,0,0] += tot_chicken_popul # <---- set total chicken population
init_val[0,3,0] += tot_duck_popul # <---- set total duck population

init_val[0,0,0] -= surveillance
init_val[0,1,0] += surveillance # chicken under surveillance 

init_val[0,0,0] -= vaccinated
init_val[0,2,0] += vaccinated # vaccinated chicken


# store the total population for each flock and each species
tot_popul = init_val[:,:,0].copy()

# then choose a bird to be exposed (symptomatic), here we assume it to be a chicken
init_val[0,0,0] -= 1
init_val[0,0,1] += 1


# this is the maximum number of events that would occur, typically the number will not be reached, but 
# for diseases that does not die out this is necessary to not fall into an infinite while loop.
max_events = 500000


We may now run the simulations:

In [46]:

######## Define update rules to be used in the Gillespie Algorithm ########

def S_to_E(a, b, current_val, symptomatic = True, tot_popul=tot_popul, beta_S=beta_S, beta_A=beta_A, p_S=p_S, p_A=p_A, num_flocks=num_flocks, num_species=num_species):
    val = 0
    for i in range(num_flocks):
        for j in range(num_species):
            val += (beta_S[i,j,a,b] * current_val[i,j,3] + beta_A[i,j,a,b] * current_val[i,j,4]) / np.sum(tot_popul[i,:])

    if symptomatic:
        val = val * current_val[a,b,0] * p_S[b]
    else:
        val = val * current_val[a,b,0] * p_A[b]
    return val

def E_to_I(a, b, current_val, symptomatic = True, tot_popul=tot_popul, sigma_S=sigma_S, sigma_A=sigma_A, num_flocks=num_flocks, num_species=num_species):
    if symptomatic:
        return current_val[a,b,1] * sigma_S[b]
    else:
        return current_val[a,b,2] * sigma_A[b]

def I_to_R(a, b, current_val, symptomatic = True, tot_popul=tot_popul, gamma_S=gamma_S, gamma_A=gamma_A, num_flocks=num_flocks, num_species=num_species):
    if symptomatic:
        return current_val[a,b,3] * gamma_S[b]
    else:
        return current_val[a,b,4] * gamma_A[b]
    

######## Gillespie Algorithm ########

def Gillespie_simu(max_events=max_events, init_val=init_val, tot_popul=tot_popul, 
                   beta_S=beta_S, beta_A=beta_A, sigma_S=sigma_S, sigma_A=sigma_A,
                   gamma_S=gamma_S, gamma_A=gamma_A, p_S=p_S, p_A=p_A, num_flocks=num_flocks, 
                   num_species=num_species):

    # initialise the event count and current values

    num_event = 0
    current_val = init_val.copy()

    # set the time and state sequence
    t = [0] + [None] * max_events
    y = [init_val] + [None] * max_events


    while (num_event < max_events) and (np.sum(current_val[:,:,1:5]) > 0): # stop the loop if: 1. maximum event number is reached, or 2. no more infections can possibly occur.
        
        num_event += 1

        ##### create an event tensor ####

        all_events = np.zeros((num_flocks, num_species, 6)) # six types of update rules in total
        for i in range(num_flocks):
            for j in range(num_species):
                all_events[i, j, 0] = S_to_E(i, j, current_val, True)
                all_events[i, j, 1] = S_to_E(i, j, current_val, False)
                all_events[i, j, 2] = E_to_I(i, j, current_val, True)
                all_events[i, j, 3] = E_to_I(i, j, current_val, False)
                all_events[i, j, 4] = I_to_R(i, j, current_val, True)
                all_events[i, j, 5] = I_to_R(i, j, current_val, False)


        # store total rate to rescale later
        tot_rate = np.sum(all_events)
        
        # do a time leap
        
        r1 = np.random.uniform()
        t[num_event] = t[num_event-1] - np.log(r1) / tot_rate
        
        # then choose events, first choose the type of events (S to E_S, S to E_A, E_S to I_S, E_A to I_A, I_S to R, or I_A to R)
        
        r2 = np.random.uniform()

        for event in range(6):
            if r2 < np.sum(all_events[:,:,0:event+1]) / tot_rate:
                type_event = event
                break

        # then choose which flock gets updated
        
        r3 = np.random.uniform()
        spec_event_rate = np.sum(all_events[:,:,type_event]) # total rate of a specific event occurring

        for i in range(num_flocks):
            if r3 < np.sum(all_events[0:i+1,:,type_event]) / spec_event_rate:
                flock_to_update = i
                break

        # finally choose which species get updated

        r4 = np.random.uniform()
        spec_event_flock_rate = np.sum(all_events[flock_to_update,:,type_event])

        for j in range(num_species):
            if r4 < np.sum(all_events[flock_to_update,0:j+1,type_event]) / spec_event_flock_rate:
                species_to_update = j
                break

        # do the updating
        if type_event == 0:
            current_val[flock_to_update, species_to_update, 0] -= 1
            current_val[flock_to_update, species_to_update, 1] += 1
        if type_event == 1:
            current_val[flock_to_update, species_to_update, 0] -= 1
            current_val[flock_to_update, species_to_update, 2] += 1
        if type_event == 2:
            current_val[flock_to_update, species_to_update, 1] -= 1
            current_val[flock_to_update, species_to_update, 3] += 1
        if type_event == 3:
            current_val[flock_to_update, species_to_update, 2] -= 1
            current_val[flock_to_update, species_to_update, 4] += 1
        if type_event == 4:
            current_val[flock_to_update, species_to_update, 3] -= 1
            current_val[flock_to_update, species_to_update, 5] += 1
        if type_event == 5:
            current_val[flock_to_update, species_to_update, 4] -= 1
            current_val[flock_to_update, species_to_update, 5] += 1

        # store the updated value

        y[num_event] = current_val.copy()

    # get rid of none value if there is any:
    t = np.array(t[0:num_event+1])
    y = np.array(y[0:num_event+1])

    return t, y


In [47]:

######## Outbreak statistics ########
def outbreak_statistics(t, y, outbreak_threshold = 5):
    # Outbreak or not
    outbreak = len([state for state in y if np.sum(state[:,:,3:5]) >= outbreak_threshold]) > 0

    # if there is an outbreak, when does it happen
    if outbreak:
        outbreak_time = t[[i for i, state in enumerate(y) if np.sum(state[:,:,3:5]) >= outbreak_threshold][0]]
    else:
        outbreak_time = None

    # Species-specific outbreak
    outbreak_indiv = [len([state for state in y if np.sum(state[:,i,3:5]) >= outbreak_threshold]) > 0 for i in range(num_species)]
    outbreak_time_indiv = [t[[j for j, state in enumerate(y) if np.sum(state[:,i,3:5]) >= outbreak_threshold][0]] if outbreak_indiv[i] else None for i in range(num_species)]
    return outbreak, outbreak_time, outbreak_indiv, outbreak_time_indiv

######## Peak size statistics ########
def peak_size(t, y):
    ######## Peak size for all flocks as a whole ########
    peak_size_whole = np.max([np.sum(state[:,:,3:5]) for state in y])
    peak_time_whole = t[np.argmax([np.sum(state[:,:,3:5]) for state in y])]

    ######## Individual species peak size ########
    peak_size_indiv = [np.amax([np.sum(state[:,i,3:5]) for state in y], axis=0) for i in range(num_species)]
    peak_time_indiv = [t[j] for j in [np.argmax([np.sum(state[:,i,3:5]) for state in y], axis=0) for i in range(num_species)]]
    return peak_size_whole, peak_time_whole, peak_size_indiv, peak_time_indiv

######## Final size and end time ########
def final_size_end_time(t, y):
    ######## Final size ########
    final_size = np.sum(y[-1,:,:,-1], axis=0)
    end_time = t[-1]
    return final_size, end_time

######## Obtain time of surveillance outcomes ########

def sentinel_outcomes(t, y, testing_period=testing_period, detection_threshold=1):

    testing_time = np.array(range(0, int(max(t)), testing_period))
    testing_index = np.zeros(len(testing_time))

    for test in range(len(testing_time)):
        i = np.searchsorted(t, testing_time[test], side='right') - 1
        testing_index[test] = i

    # Obtain the result of all testing:
    testing_result = [np.sum(y[int(i), 0, 1, 3:5]) for i in testing_index] # test int(i), flock 0, sentinel chicken, I
    detection_time = next((testing_time[i] for i, x in enumerate(testing_result) if x >= detection_threshold), None)

    return detection_time   

def random_sample_outcomes(t, y, surveillance=surveillance, testing_period=testing_period, detection_threshold=1):

    testing_time = np.array(range(0, int(max(t)), testing_period))
    testing_index = np.zeros(len(testing_time))

    for test in range(len(testing_time)):
        i = np.searchsorted(t, testing_time[test], side='right') - 1
        testing_index[test] = i

    # Obtain the result of all testing:
    testing_result = [np.random.binomial(surveillance, np.sum(y[int(i), 0, 0:2, 3:5]) / np.sum(y[int(i), 0, 0:3, :])) for i in testing_index] # a binomial draw based on the proportion of infected birds
    detection_time = next((testing_time[i] for i, x in enumerate(testing_result) if x >= detection_threshold), None)

    return detection_time

In [ ]:
######## Collect all statistics for a number of simulations ########
def mass_simu(num_simu, max_events=max_events, init_val=init_val, tot_popul=tot_popul, 
              beta_S=beta_S, beta_A=beta_A, sigma_S=sigma_S, sigma_A=sigma_A,
              gamma_S=gamma_S, gamma_A=gamma_A, p_S=p_S, p_A=p_A, num_flocks=num_flocks, 
              num_species=num_species):
    t_mass_simu = []
    y_mass_simu = []

    param = max_events, init_val, tot_popul, beta_S, beta_A, sigma_S, sigma_A, gamma_S, gamma_A, p_S, p_A, num_flocks, num_species
    result = list(map(lambda p: Gillespie_simu(*p), [param]*num_simu))

    t_mass_simu, y_mass_simu = zip(*result)
    return list(t_mass_simu), list(y_mass_simu)

def mass_outbreak_statistics(t_mass_simu, y_mass_simu, outbreak_threshold=5):
    simu = [(t_mass_simu[i], y_mass_simu[i], outbreak_threshold) for i in range(len(t_mass_simu))]
    result = list(map(lambda s: outbreak_statistics(*s), simu))

    mass_outbreak, mass_outbreak_time, mass_outbreak_indiv, mass_outbreak_time_indiv = zip(*result)

    return np.array(mass_outbreak), np.array(mass_outbreak_time), np.array(mass_outbreak_indiv), np.array(mass_outbreak_time_indiv)

def mass_peak_size(t_mass_simu, y_mass_simu):
    simu = [(t_mass_simu[i], y_mass_simu[i]) for i in range(len(t_mass_simu))]
    result = list(map(lambda s: peak_size(*s), simu))

    mass_peak_size_whole, mass_peak_time_whole, mass_peak_size_indiv, mass_peak_time_indiv = zip(*result)

    return np.array(mass_peak_size_whole), np.array(mass_peak_time_whole), np.array(mass_peak_size_indiv), np.array(mass_peak_time_indiv)

def mass_final_size_end_time(t_mass_simu, y_mass_simu):
    simu = [(t_mass_simu[i], y_mass_simu[i]) for i in range(len(t_mass_simu))]
    result = list(map(lambda s: final_size_end_time(*s), simu))

    mass_final_size, mass_end_time = zip(*result)

    return np.array(mass_final_size), np.array(mass_end_time)

def mass_sentinel_outcomes(t_mass_simu, y_mass_simu):
    simu = [(t_mass_simu[i], y_mass_simu[i]) for i in range(len(t_mass_simu))]
    result = list(map(lambda s: sentinel_outcomes(*s), simu))
    
    mass_detection_time = np.array(result)

    return mass_detection_time
    
def mass_random_sample_outcomes(t_mass_simu, y_mass_simu):
    simu = [(t_mass_simu[i], y_mass_simu[i]) for i in range(len(t_mass_simu))]
    result = list(map(lambda s: random_sample_outcomes(*s), simu))
    
    mass_detection_time = np.array(result)

    return mass_detection_time


######## Run the simulation ########
t_mass_simu, y_mass_simu = mass_simu(num_simu)

mass_outbreak, mass_outbreak_time, mass_outbreak_indiv, mass_outbreak_time_indiv = mass_outbreak_statistics(t_mass_simu, y_mass_simu)
mass_peak_size_whole, mass_peak_time_whole, mass_peak_size_indiv, mass_peak_time_indiv = mass_peak_size(t_mass_simu, y_mass_simu)
mass_final_size, mass_end_time = mass_final_size_end_time(t_mass_simu, y_mass_simu)
mass_detection_time_sentinel = mass_sentinel_outcomes(t_mass_simu, y_mass_simu)
mass_detection_time_random = mass_random_sample_outcomes(t_mass_simu, y_mass_simu)

######## Create a csv file to store the results ########
df = pd.DataFrame({'Outbreak': mass_outbreak,
                   'Outbreak Time': mass_outbreak_time,
                   'Outbreak Chicken': mass_outbreak_indiv[:, 0],
                   'Outbreak Duck': mass_outbreak_indiv[:, 3], 
                   'Outbreak Time Chicken': mass_outbreak_time_indiv[:, 0], 
                   'Outbreak Time Duck': mass_outbreak_time_indiv[:, 3], 
                   'Peak Size Whole': mass_peak_size_whole, 
                   'Peak Time Whole': mass_peak_time_whole, 
                   'Peak Size Chicken': np.sum(mass_peak_size_indiv[:, :3], axis=1), 
                   'Peak Time Chicken': np.sum(mass_peak_time_indiv[:, :3], axis=1), 
                   'Peak Size Duck': mass_peak_size_indiv[:, 3], 
                   'Peak Time Duck': mass_peak_time_indiv[:, 3], 
                   'Final Size Chicken': np.sum(mass_final_size[:, :3], axis=1), 
                   'Final Size Duck': mass_final_size[:, 3], 
                   'End Time': mass_end_time,
                   'Detection Time Sentinel': mass_detection_time_sentinel,
                   'Detection Time Random': mass_detection_time_random})


df.to_csv(f'Results_vaccinated_{vaccinated}_ducks_{tot_duck_popul}_dssi_{different_species_symptomatic_infection_rate}_dsai_{different_species_asymptomatic_infection_rate}.csv', index=False)

In [49]:
df

,Outbreak,Outbreak Time,Outbreak Chicken,Outbreak Duck,Outbreak Time Chicken,Outbreak Time Duck,Peak Size Whole,Peak Time Whole,Peak Size Chicken,Peak Time Chicken,Peak Size Duck,Peak Time Duck,Final Size Chicken,Final Size Duck,End Time,Detection Time Sentinel,Detection Time Random
0,True,6.953381,True,True,7.830016,8.142052,1298.0,29.586705,512.0,55.847586,791.0,29.593064,2622.0,2826.0,72.809239,21,14
1,True,4.978606,True,True,4.978606,13.234473,1248.0,39.267420,537.0,78.086610,732.0,39.266106,2639.0,2832.0,76.642692,28,28
2,True,6.601592,True,True,6.601592,8.862735,1254.0,32.001471,525.0,59.459761,747.0,30.864729,2666.0,2832.0,78.758518,21,21
3,True,2.589568,True,True,2.755409,8.163789,1308.0,29.000126,547.0,60.550603,778.0,27.840773,2688.0,2835.0,76.353612,14,21
4,False,None,False,False,None,None,1.0,0.191861,1.0,0.191861,0.0,0.000000,1.0,0.0,0.514711,None,None
5,True,9.94984,True,True,11.920536,12.176217,1259.0,38.452306,509.0,73.576461,761.0,38.691249,2681.0,2813.0,83.382358,35,28
6,True,5.177986,True,True,13.36409,9.241007,1289.0,33.390874,523.0,71.442783,774.0,33.126182,2677.0,2859.0,84.166221,7,28
7,True,9.650282,True,True,10.553244,12.118944,1262.0,35.164191,511.0,75.435104,760.0,35.045478,2678.0,2828.0,75.452106,21,21
8,True,5.60207,True,True,5.61255,7.551184,1382.0,29.432315,584.0,58.989672,808.0,29.270897,2715.0,2832.0,71.974050,14,21
9,False,None,False,False,None,None,1.0,2.682136,1.0,2.682136,0.0,0.000000,1.0,0.0,3.120025,None,None
